In [1]:
from pims import ND2Reader_SDK

In [2]:
directory = "/home/georgeos/Dropbox (Cambridge University)/Nikon Nd2 Backup_Bakshi-Lab/gh464/MI13TnaAGFP_exit_dyn/ND2/MI13TnaAGFP_exit_dyn.nd2"
frames =  ND2Reader_SDK(directory)

/home/georgeos/miniconda3/envs/nd2_extractor/lib/python3.10/site-packages/pims/base_frames.py:478: UserWarning: Please call FramesSequenceND.__init__() at the start of thethe reader initialization.
  warn("Please call FramesSequenceND.__init__() at the start of the"


In [3]:
type(frames.metadata)

dict

In [14]:
frames.

39

In [5]:
import yaml

In [7]:
with open('metadata.yml', 'w') as outfile:
    yaml.dump(frames.metadata, outfile, default_flow_style=False)

In [8]:
frames.metadata

{'width': 2304,
 'width_bytes': 13824,
 'height': 2304,
 'components': 3,
 'bitsize_memory': 16,
 'bitsize_significant': 16,
 'sequence_count': 11271,
 'tile_width': 2304,
 'tile_height': 2304,
 'compression': None,
 'compression_quality': 4294967197,
 'plane_count': 3,
 'angle': -3.1057421608167726,
 'calibration_um': 0.065,
 'time_start_jdn': 2459838.2996869446,
 'time_start': datetime.datetime(2022, 9, 15, 20, 11, 32, 952012),
 'time_start_utc': datetime.datetime(2022, 9, 15, 19, 11, 32, 952012),
 'objective': 'Plan Apo λ 100x Oil Ph3 DM',
 'magnification': -1.0,
 'NA': 1.45,
 'refractive_index1': 1.515,
 'refractive_index2': 1.515,
 'pinhole': 0.0,
 'zoom': 1.0,
 'projective_mag': -1.0,
 'image_type': 'normal',
 'z_home': None,
 'frame_rate': 0.00555577647149626,
 'plane_0': {'components': 1,
  'rgb_value': (0.07058823529411765, 1.0, 0.0),
  'name': 'Green',
  'oc': 'Green',
  'emission_nm': 515.0},
 'plane_1': {'components': 1,
  'rgb_value': (1.0, 1.0, 1.0),
  'name': 'PC',
  'oc